# Configuração e Carregamento de Dados

In [21]:
!pip install pyspark # nstala a biblioteca PySpark no ambiente de execução

In [22]:
import pyspark # Importa a biblioteca principal do PySpark
from pyspark.sql import SparkSession # mporta a classe SparkSession, o ponto de entrada para o Spark.
from pyspark.sql.functions import * # Importa todas as funções de manipulação de colunas e agregação (embora a maioria das operações nesta seção use SQL puro)

In [23]:
from google.colab import drive # Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive para permitir o acesso aos arquivos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [25]:
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True) # Lê o DataFrame de produtos
vendedores =  spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/vendedores.csv", header=True, inferSchema=True) # Lê o DataFrame de vendedores
clientes = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True) # Lê o DataFrame de clientes
itens_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de itens_pedido
pagamentos_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pagamentos_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de pagamentos_pedido
avaliacoes_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/avaliacoes_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de avaliacoes_pedido
pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True) # Lê o DataFrame de pedidos

In [26]:
# Exibe as 2 primeiras linhas de cada um dos 7 DataFrames carregados para verificação
produtos.show(2)
vendedores.show(2)
clientes.show(2)
itens_pedido.show(2)
pagamentos_pedido.show(2)
avaliacoes_pedido.show(2)
pedidos.show(2)

+--------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|       perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|            artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
+--------------------+-----------------+--------------------

# Criação de Views Temporárias (SQL)

In [27]:
produtos.createOrReplaceTempView('produtos_temp') # Cria uma view temporária chamada produtos_temp que pode ser consultada via Spark SQL. Se a view já existir, ela será substituída
# Cria views temporárias para todos os outros 6 DataFrames
vendedores.createOrReplaceTempView('vendedores_temp')
clientes.createOrReplaceTempView('clientes_temp')
itens_pedido.createOrReplaceTempView('itens_pedido_temp')
pagamentos_pedido.createOrReplaceTempView('pagamentos_pedido_temp')
avaliacoes_pedido.createOrReplaceTempView('avaliacoes_pedido_temp')
pedidos.createOrReplaceTempView('pedidos_temp')

In [28]:
spark.catalog.listDatabases() # Lista todos os bancos de dados disponíveis no catálogo do Spark

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/content/spark-warehouse')]

In [29]:
spark.catalog.listTables() # Lista todas as tabelas (incluindo as views temporárias criadas) no catálogo do Spark

[Table(name='avaliacoes_pedido_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='clientes_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='itens_pedido_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='pagamentos_pedido_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='pedidos_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='produtos_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='vendedores_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

# Consultas SQL Complexas e Joins

In [30]:
# Seleciona informações de itens de pedido e produtos, realizando um JOIN entre as views itens_pedido_temp e produtos_temp na coluna id_produto
itens_pedido_produto_df = spark.sql("""
   SELECT pr.id_produto, pr.categoria_produto, ip.valor_frete, ip.preco
   FROM itens_pedido_temp ip
   JOIN produtos_temp pr ON ip.id_produto = pr.id_produto
""")

itens_pedido_produto_df.printSchema() # mprime o esquema do DataFrame resultante do primeiro join
itens_pedido_produto_df.show(5) # Exibe as 5 primeiras linhas do DataFrame resultante

root
 |-- id_produto: string (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- valor_frete: double (nullable = true)
 |-- preco: double (nullable = true)

+--------------------+------------------+-----------+-----+
|          id_produto| categoria_produto|valor_frete|preco|
+--------------------+------------------+-----------+-----+
|4244733e06e7ecb49...|        cool_stuff|      13.29| 58.9|
|e5f2d52b802189ee6...|          pet_shop|      19.93|239.9|
|c777355d18b72b67a...|  moveis_decoracao|      17.87|199.0|
|7634da152a4610f15...|        perfumaria|      12.79|12.99|
|ac6c3623068f30de0...|ferramentas_jardim|      18.14|199.9|
+--------------------+------------------+-----------+-----+
only showing top 5 rows



In [31]:
# Define uma query que une avaliacoes_pedido_temp e itens_pedido_temp para calcular a média de avaliação (AVG(nota_avaliacao)) por id_produto.
query = 'SELECT ip.id_produto, AVG(ap.nota_avaliacao) AS media_avaliacao FROM avaliacoes_pedido_temp ap JOIN itens_pedido_temp ip ON ap.id_pedido = ip.id_pedido GROUP BY ip.id_produto'
avaliacao_produto_df = spark.sql(query) # Executa a query SQL e armazena o resultado em um novo DataFrame
avaliacao_produto_df.show() # Exibe o DataFrame de avaliação média por produto

+--------------------+------------------+
|          id_produto|   media_avaliacao|
+--------------------+------------------+
|0b0172eb0fd18479d...|             3.875|
|42a2bd596fda1baef...|               4.0|
|460a66fcc404a3d73...|3.8333333333333335|
|30360c8b0b2ac6918...| 4.666666666666667|
|13b4ff901d43edec6...|               5.0|
|290ada89b05e1dca2...|               5.0|
|7724696de32f44179...|               4.0|
|abe236a52dbc43e90...| 4.111111111111111|
|e85cdca8790ea0026...|               3.8|
|35bc6c77029697004...|3.8947368421052633|
|878699846fa5ea02a...|               1.0|
|2f9c2888168b8c2d8...|3.3333333333333335|
|75f3ef6a5cb0f2d5a...|1.6666666666666667|
|3f1a741cf55913844...|              4.25|
|526e24e5c44ede53f...| 4.333333333333333|
|be7f0912ba0112670...|               4.2|
|18d87751433e4198e...|3.6666666666666665|
|b23cfa104a0f29745...|               3.0|
|5a57a59c44429be19...|               4.4|
|ed464125465d8ab04...|3.2222222222222223|
+--------------------+------------

In [32]:
detalhe_produto_df = avaliacao_produto_df.join(itens_pedido_produto_df, 'id_produto') # Combina o resultado da média de avaliação (avaliacao_produto_df) com o resultado do primeiro join (itens_pedido_produto_df) usando o id_produto como chave
detalhe_produto_df.show() # Exibe o DataFrame final com todos os detalhes do produto, incluindo preço, frete e avaliação média.

+--------------------+------------------+--------------------+-----------+-----+
|          id_produto|   media_avaliacao|   categoria_produto|valor_frete|preco|
+--------------------+------------------+--------------------+-----------+-----+
|0b0172eb0fd18479d...|             3.875|         eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|             3.875|         eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|             3.875|         eletronicos|      17.63|24.89|
|42a2bd596fda1baef...|               4.0|     cama_mesa_banho|      17.88| 89.9|
|460a66fcc404a3d73...|3.8333333333333335|utilidades_domest...|      12.84|24.98|
|42a2bd596fda1baef...|               4.0|     cama_mesa_banho|      25.05| 99.9|
|30360c8b0b2ac6918...| 4.666666666666667|  relogios_presentes|      19.43|189.9|
|13b4ff901d43edec6...|               5.0|     cama_mesa_banho|      64.49|630.0|
|13b4ff901d43edec6...|               5.0|     cama_mesa_banho|      64.49|630.0|
|290ada89b05e1dca2...|      

# Persistência e Salvamento de Dados

In [ ]:
import shutil

# Salva o DataFrame no Google Drive no formato Parquet, substituindo (overwrite) se já existir
detalhe_produto_df.write.mode('overwrite').parquet('/content/drive/MyDrive/Material de apoio - M27/output/detalhe_produto_parquet') # salva no google drive

# Salva o DataFrame como uma tabela gerenciada chamada detalhe_produto no warehouse local do Spark, permitindo consultas SQL diretas
detalhe_produto_df.write.mode('overwrite').option('header', 'true').saveAsTable('detalhe_produto') # salva localmente no warehouse do Spark

# Salva o DataFrame como uma tabela externa chamada detalhe_produto_table, mas armazena os dados fisicamente no caminho especificado no Google Drive
detalhe_produto_df.write.mode('overwrite').option('path', '/content/drive/MyDrive/Material de apoio - M27/output/table_produto/').saveAsTable('detalhe_produto_table') # salva no google drive (atenção)

In [54]:
query = 'select * from detalhe_produto' # Define uma query para ler a tabela salva localmente no warehouse do Spark
teste_detalhe_produto = spark.sql(query) # Executa a query para ler a tabela salva
teste_detalhe_produto.show() # Exibe o conteúdo da tabela localmente salva

+--------------------+---------------+-----------------+-----------+-----+
|          id_produto|media_avaliacao|categoria_produto|valor_frete|preco|
+--------------------+---------------+-----------------+-----------+-----+
|0b0172eb0fd18479d...|          3.875|      eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      22.67| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.92| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      26.89| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|       8.11|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.11| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      12.48|24.89|
|0b0172eb0fd18479d...|   

In [55]:
# Lê a tabela salva no Google Drive, especificando o caminho e o nome da tabela
detalhe_produto_table_df = spark.read.option('path', '/content/drive/MyDrive/Material de apoio - M27/output/table_produto/-').table('detalhe_produto_table')

In [56]:
detalhe_produto_table_df.show() # Exibe o conteúdo da tabela salva no Drive

+--------------------+---------------+-----------------+-----------+-----+
|          id_produto|media_avaliacao|categoria_produto|valor_frete|preco|
+--------------------+---------------+-----------------+-----------+-----+
|0b0172eb0fd18479d...|          3.875|      eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      22.67| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.92| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      26.89| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|       8.11|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.11| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      12.48|24.89|
|0b0172eb0fd18479d...|   

# Análise Temporal (SQL)

In [57]:
# Extrai o ano (YEAR()) e o mês (MONTH()) da coluna data_compra_pedido. Em seguida, agrupa por ano e mês para contar o total de pedidos (COUNT(*)), ordenando do ano mais recente para o mais antigo e depois pelo mês
spark.sql("""
  SELECT
          YEAR(data_compra_pedido) AS compra_anual,
          MONTH(data_compra_pedido) AS compra_mensal,
          COUNT(*) AS count
  FROM
      pedidos_temp
  GROUP BY
          compra_anual,
          compra_mensal
   ORDER BY
           compra_anual DESC,
           compra_mensal
""").show()

+------------+-------------+-----+
|compra_anual|compra_mensal|count|
+------------+-------------+-----+
|        2018|            1| 7269|
|        2018|            2| 6728|
|        2018|            3| 7211|
|        2018|            4| 6939|
|        2018|            5| 6873|
|        2018|            6| 6167|
|        2018|            7| 6292|
|        2018|            8| 6512|
|        2018|            9|   16|
|        2018|           10|    4|
|        2017|            1|  800|
|        2017|            2| 1780|
|        2017|            3| 2682|
|        2017|            4| 2404|
|        2017|            5| 3700|
|        2017|            6| 3245|
|        2017|            7| 4026|
|        2017|            8| 4331|
|        2017|            9| 4285|
|        2017|           10| 4631|
+------------+-------------+-----+
only showing top 20 rows



# Encerramento da Sessão

In [58]:
spark.stop() # Exibe o resultado da contagem de pedidos por ano e mês